# UFSM
## Engenharia Acústica 
## Processamento Digital de Sinais II
### Trabalho Integrado: rotina de medição

In [1]:
# Carregando as bibliotecas que utilizaremos
import pytta

#### Organização dos dados medidos
 
O tipo de dado Struct, do MATLAB, funciona de forma semelhante ao Dictionary do Python. Vamos organizar as variáveis com dicionários dentro de dicionários.

Sugiro que os dados sejam organizados nos seguintes níveis de informação e sequência:
1. Configuração fonte receptor; e.g. S1R1;
2. Sinal de excitação: sweep, música ou fala;
3. Sinal gravado: biauricular, monofônico

É notável que no momento da medição os dados não serão adquiridos nessa sequência. Uma parte do trabalho da nossa rotina de medição é pegar os dados medidos na última tomada e inseri-los no lugar correto do nosso dicionário de medição. Pois a cada tomada o número de posições medidas de receptor  mudará, como já vimos na matriz de teste (e.g. tomada 1 com S1R1 - biauricular, S1R2 - centro da cabeça - e S1R3 - centro do cabeça; tomada 'n' somente com S2R5 - biauricular).

#### Exemplo de dicionário de medição

In [ ]:
SnRn = {'sweep':{},'music':{},'speech':{}}  # Cria um dicionário referente as medições de uma configuração 
                                            # fonte-receptor qualquer

measurement = {} # Cria o dicionário vazio que conterá todos os níveis de informação do nosso dia de medição

asdasdasdasd =asdasdasd;


In [ ]:
RecMeasure = pytta.generate.measurement(kind = 'rec',
                                        lengthDomain = 'samples',
                                        fftDegree = 14,
                                        samplingRate = pytta.default.samplingRate,
                                        freqMin = 20,
                                        freqMax = 20000,
                                        device = [0,1],
                                        inChannel = [1],
                                        comment = 'RecMeasure test')